# Вейвлет-сжатие изображения методом Хаара

## Импорт библиотек

In [9]:
import cv2
import numpy as np
import os
from collections import Counter

## Пункт 1: Сохранение монохромного изображения

In [10]:
img_path = 'sar_1_gray.jpg'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

np.savetxt('pixel_image.txt', img, fmt='%d')
pixel_storage_size = os.path.getsize('pixel_image.txt')

## Пункт 2: Вейвлет-преобразование Хаара

In [11]:
def haar_transform(image):
    rows, cols = image.shape
    image_float = image.astype(np.float64)
    
    # Преобразование по строкам
    row_transform = np.zeros_like(image_float)
    for i in range(rows):
        for j in range(0, cols - 1, 2):
            row_transform[i, j // 2] = (image_float[i, j] + image_float[i, j + 1]) / 2
            row_transform[i, (j // 2) + cols // 2] = (image_float[i, j] - image_float[i, j + 1]) / 2
    
    # Преобразование по столбцам
    result = np.zeros_like(row_transform)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            result[i // 2, j] = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            result[(i // 2) + rows // 2, j] = (row_transform[i, j] - row_transform[i + 1, j]) / 2
    
    LL = result[:rows // 2, :cols // 2]
    LH = result[:rows // 2, cols // 2:]
    HL = result[rows // 2:, :cols // 2]
    HH = result[rows // 2:, cols // 2:]
    
    return LL, LH, HL, HH

LL, LH, HL, HH = haar_transform(img)
print("Вейвлет-преобразование выполнено")

Вейвлет-преобразование выполнено


## Пункт 3: Квантование высокочастотных компонент

In [12]:
def quantize(coeffs, n_quants):
    min_val = np.min(coeffs)
    max_val = np.max(coeffs)
    step = (max_val - min_val) / n_quants
    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized

LH_q = quantize(LH, 4)
HL_q = quantize(HL, 4)
HH_q = quantize(HH, 4)

print("Квантование выполнено")

Квантование выполнено


## Пункт 4: RLE-кодирование и сохранение

In [13]:
def run_length_encode(data):
    encoded = []
    for value, count in Counter(data.flatten()).items():
        encoded.append((value, count))
    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

print("RLE-кодирование выполнено")

RLE-кодирование выполнено


## Сохранение результатов в файл

In [14]:
with open('haar_image.txt', 'w') as f:
    np.savetxt(f, LL, fmt='%d')
    
    f.write("LH:\n")
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    
    f.write("HL:\n")
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")

    f.write("HH:\n")
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")


## Сравнение объемов памяти

In [15]:
haar_storage_size = os.path.getsize('haar_image.txt')

print("РЕЗУЛЬТАТЫ СЖАТИЯ")
print(f"Исходный размер: {pixel_storage_size} байт")
print(f"Размер после сжатия: {haar_storage_size} байт")
print(f"Коэффициент сжатия: {pixel_storage_size / haar_storage_size:.2f}")
print(f"Экономия памяти: {(1 - haar_storage_size / pixel_storage_size) * 100:.1f}%")

РЕЗУЛЬТАТЫ СЖАТИЯ
Исходный размер: 764822 байт
Размер после сжатия: 190893 байт
Коэффициент сжатия: 4.01
Экономия памяти: 75.0%
